In [ ]:
import os, sys
import random
import time
import imutils
import math
import numpy as np
import pandas as pd
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
from matplotlib.patches import Polygon

### Function to implement tracking on one video sequence

In [ ]:
# this runs once per clip per tracker per resize value
# lists declared and modified here pertain to one clip
def run_track(tracker_name, tracker_inst, video_name, video_file, frames_df, resize):
    '''
    Runs a specified tracking algorithm on a specified video \
    file and evaluates tracker performance by comparing system \
    output to specified ground truth bounding boxes.
    
    Parameters:
    
    tracker_name (str): short name of tracking algo
    tracker_inst: instantiation of tracker class
    video_file: (str): path to video file
    frames_df (pd.DataFrame): DF of ground truth objects in video clip
    resize (int): fraction to which video should be resized (1, 2 or 4). 1 = unchanged (Def), 2 = 1/2, 4 = 1/4
    '''
    
    # CONSTANTS
    THRESHOLD_IOU, THRESHOLD_DIST = 0.4, 0.2
    
    # initialise vars
    frame_count = frames_df['frame'].min() # works fine
    fn_count, fp_count, fp_thresh_count, tp_count, tn_count, covered_count = 0, 0, 0, 0, 0, 0 # seem to work fine
    tracking_fps = 0 # seems to work fine
    iou_list, dist_list = [], [] # seems to work fine
    tracker = None # presumably fine
    writer = None # works
    draw = False # works
    
    # Initialize the video stream and pointer to output video file
    vs = cv2.VideoCapture(video_file)
    
    # start clock for FPS
    tracker_time = start_timer()
    
    while True:
        # read the next frame from the file
        grabbed, frame = vs.read()
        
        # if no frame grabbed, reached end of clip
        if not grabbed:
            print ('[INFO] End of clip')
            break
        
        # boolean for whether there is a ground truth object in the frame
        object_exists = (frames_df['frame'] == frame_count).any()

        if resize is not 0:
            frame = cv2.resize(frame, (resize, int(resize*(9/16))))
            
        # will resize the frame for faster processing
        new_width = frame.shape[1]
        new_height = frame.shape[0]
        
        # number of rows in frames determine number of objects, N_objects
        N_objects = len(frames_df.index)
        
        # bb for ground truth obj (relative values)
        gt_bb = (frames_df.loc[frames_df['frame'] == frame_count].squeeze()[1:])*(new_width, new_height, new_width, new_height) if object_exists else 0
        print('[INFO] gt_bb is: \n')
        
        # initialize the writer
        if writer is None:
            print()
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            fps = vs.get(5)
            # writer = cv2.VideoWriter('test_writer.mp4', fourcc, fps, (frame.shape[1], frame.shape[0]), True)
            writer = cv2.VideoWriter('./output/' + video_name + '_' + tracker_name + \
                                     '_' + str(new_width) + '.mp4', fourcc, fps, \
                                     (frame.shape[1], frame.shape[0]), True)
        
        if tracker is None:
            #print ('[INFO] Create tracker instance')
            # create tracker object
            tracker = tracker_inst
            
            # initialise with bounding box
            tracker.init(frame, tuple(gt_bb))
            
        if tracker is not None:
            # update the tracker and grab the tracked object
            tracking, trk_bb = tracker.update(frame)
            
            # will need modifications for trackers outside of OpenCV-8
            if tracking is None:
                #print ('[INFO] tracking is none')
                if ~object_exists:
                    # true negative
                    tn_count += 1
                    
                elif object_exists:
                    # false negative
                    fn_count += 1

            elif tracking:
                draw = True
                if object_exists:
                    iou_t, dist_t = compute_iou_dist(gt_bb, trk_bb) # pass them in (x1, y1, w, h) format
                    
                    # completeness metrics
                    if iou_t >= THRESHOLD_IOU:
                        covered_count += 1
                    
                    if dist_t < THRESHOLD_DIST:
                        # false positive
                        fp_thresh_count += 1
                        
                    elif dist_t >= THRESHOLD_DIST:
                        # true positive
                        tp_count += 1
                        
                    iou_list.append(iou_t)
                    dist_list.append(dist_t)
                    
                elif ~object_exists:
                    # false positive
                    fp_count += 1
                
                # tracking taken place, so draw frame
                if draw:
                    p1 = (int(trk_bb[0]), int(trk_bb[1]))
                    p2 = (int(trk_bb[0] + trk_bb[2]), int(trk_bb[1] + trk_bb[3]))
                    cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        
        # increment frame counter
        frame_count += 1
        
        # write the frame to disk
        if writer is not None:
            writer.write(frame)
    
    # finalise FPS
    tracking_fps = stop_and_report(frame_count, tracker_time)
    
    if writer is not None:
        writer.release()
        
    # print('[INFO] Res at end of loop: ' + str(frame.shape[:2]))
        
    # call metrics function - returns dictionary of metrics
    metrics = compute_metrics(fp_thresh_count, fp_count, tp_count, tn_count, fn_count, \
                              covered_count, dist_list, iou_list, N_objects, frame_count, \
                              tracking_fps, clip_name, str(new_width), tracker_name)
    
    return metrics

In [ ]:
def start_timer():
    return time.time()

In [ ]:
def stop_and_report(frames, start_time):
    return frames/(time.time() - start_time)

## Evaluation Metrics
This section defines functions to compute the various evaluation metrics detailed in Chapter 3 in the report. The metrics to compute are as follows:
- Recall: Correctly matched detections as proportion of total ground truth objects (of a sequence).
- Precision/N-SODA: Correctly matched detections as a proportion of total detections (of a sequence).
- FAF: Number of false alarms (incorrect detections) per frame averaged over a sequence.
- SODP: Average overlap between ground truth and system output.
- SOTA: Combines false negatives and false positives without weighting factors.
- SOTP: Average distance between centroids of ground truth and system output.
- TDE: Distance beetween the ground-truth annotation and the tracking result.
- MT: The ground-truth trajectory is covered by the tracker output for more than 80% of its length.
- ML: The ground-truth trajectory is covered by the tracker output for less than 20% of its length.
- PT: The ground-truth trajectory is covered by the tracker output for between 20% and 80% of its length.
- FM: Number of times that a ground-truth trajectory is interrupted in the tracking result, normalised over sequence.
- RS: Ratio of tracks which correctly recover from short term occlusion.
- RL: Ratio of tracks which correctly recover from long term occlusion.

### Terminology
- **Success**: aka True Positive. Overlap between ground truth and tracker hypothesis is non-zero and within threshold.
- **Success**: aka True Negative. No object and no hypothesis.
- **Miss**: aka False Negative. No hypothesis from tracker.
- **False Positive**: Overlap between tracker output and ground truth is less than threshold.
- **Distance**: Euclidean distance between centroids of tracker output and ground truth.
- **Covered**: Overlap exceeds THRESHOLD_IOU

### Function to Compute Each Metric

#### Intersection over Union (IoU) and Distance Between Centroids

In [ ]:
def compute_iou_dist(box_a, box_b):
    
    # compute centroids (x1 + half width, y1 + half height)
    centroid_a = (box_a[0] + 0.5*box_a[2], box_a[1] + 0.5*box_a[3])
    centroid_b = (box_b[0] + 0.5*box_b[2], box_b[1] + 0.5*box_b[3])
    
    # (x1, y1, x1+w, y1+h) -> (x1, y1, x2, y2)
    box_a = (box_a[0], box_a[1], box_a[0] + box_a[2], box_a[1] + box_a[3])
    box_b = (box_b[0], box_b[1], box_b[0] + box_b[2], box_b[1] + box_b[3])
    
    # determine the (x, y)-coordinates of the intersection rectangle
    x_a = max(box_a[0], box_b[0])
    y_a = max(box_a[1], box_b[1])
    x_b = min(box_a[2], box_b[2])
    y_b = min(box_a[3], box_b[3])

    # compute the area of intersection rectangle
    area_overlap = max(0, x_b - x_a + 1) * max(0, y_b - y_a + 1)

    # compute the area of both the prediction and ground-truth rectangles
    area_box_a = (box_a[2] - box_a[0] + 1) * (box_a[3] - box_a[1] + 1)
    area_box_b = (box_b[2] - box_b[0] + 1) * (box_b[3] - box_b[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = area_overlap / float(area_box_a + area_box_b - area_overlap)
    
    # distance is hypotenuse of two centroid coordinates
    dist = math.hypot(centroid_b[0] - centroid_a[0], centroid_b[1] - centroid_a[1])

    # return the intersection over union value
    return iou, dist

#### 1. SOTP - Single Object Tracking Precision, or Average Distance
Average distance between boxes only on frames with both an object and a hypothesis, i.e.:
- `fp_thresh_count`
- `tp_count`

`dist_t` should be forced to zero for frames with any results outside of the above (fp, tn, fn).

In [ ]:
def metric_sotp(dist_list, N_match):
    total_dist = sum(dist_list)
    return (total_dist/N_match) if N_match != 0 else np.nan

#### 2. SOTA

In [ ]:
def metric_sota(fp_thresh_count, fp_count, fn_count, N_objects):
    return 1 - (fp_thresh_count + fp_count + fn_count)/(N_objects)

#### 3. Recall

In [ ]:
def metric_recall(tp_count, N_objects):
    return tp_count/N_objects

#### 4. Precision

In [ ]:
def metric_precision(tp_count, fp_thresh_count, fp_count):
    return tp_count/(tp_count + fp_count + fp_thresh_count)

#### 5. FPF, False Positives per Frame

In [ ]:
def metric_fpf(fp_count, fp_thresh_count, N_frames):
    return (fp_count + fp_thresh_count)/N_frames

#### 6. SODP, Single Object Detection Precision

In [ ]:
def metric_sodp(iou_list, N_match):
    total_iou = sum(iou_list)
    return (total_iou/N_match)

#### 7. MT

In [ ]:
def metric_mt(completeness):
    return 1 if completeness >= 0.8 else 0

#### 8. PT

In [ ]:
def metric_pt(completeness):
    return 1 if (completeness > 0.2) and (completeness < 0.8) else 0

#### 9. ML

In [ ]:
def metric_ml(completeness):
    return 1 if completeness <= 0.2 else 0

#### 10. FM, Fragmentation

In [ ]:
def metric_fm(fp_thresh_count, fp_count, fn_count, N_objects):
    return (fp_thresh_count + fp_count + fn_count)/(N_objects)

### Metrics Function
This function calculates all metrics and returns them in a dictionary

In [ ]:
def compute_metrics(fp_thresh_count, fp_count, tp_count, tn_count, fn_count, \
                    covered_count, dist_list, iou_list, N_objects, N_frames, \
                    tracking_fps, clip_name, width, tracker_name):
    
    N_match = fp_thresh_count + tp_count
    completeness = covered_count/N_match if N_match != 0 else 0
    
    sotp = metric_sotp(dist_list, N_match)
    sota = metric_sota(fp_thresh_count, fp_count, fn_count, N_objects)
    recall = metric_recall(tp_count, N_objects)
    precision = metric_precision(tp_count, fp_thresh_count, fp_count)
    fpf = metric_fpf(fp_count, fp_thresh_count, N_frames)
    sodp = metric_sodp(iou_list, N_match)
    mt = metric_mt(completeness)
    pt = metric_pt(completeness)
    ml = metric_ml(completeness)
    fm = metric_fm(fp_thresh_count, fp_count, fn_count, N_objects)
    
    results_dict = {'Clip': clip_name,
                    'Res': width,
                    'Tracker': tracker_name,
                    'SOTP': np.round(sotp,4),
                    'SOTA': np.round(sota,4),
                    'Recall': np.round(recall,4),
                    'Precision': np.round(precision,4),
                    'FPF': np.round(fpf, 4),
                    'SODP': np.round(sodp,4),
                    'MT': mt,
                    'PT': pt,
                    'ML': ml,
                    'Completeness': np.round(completeness,2),
                    'FM': np.round(fm,2),
                    'FPS': np.round(tracking_fps,2),
                    'fp_count': fp_count,
                    'fp_thresh_count': fp_thresh_count,
                    'tp_count': tp_count,
                    'fn_count': fn_count,
                    'tn_count': tn_count,
                    'N_objects': N_objects,
                    'N_frames': N_frames,
                    'N_match': N_match}
    
    return results_dict

### List of trackers to run through for each video file

In [ ]:
# trackers = {'boosting': cv2.TrackerBoosting_create(),
#                 'mil': cv2.TrackerMIL_create(),
#                 'kcf': cv2.TrackerKCF_create(),
#                 'tld': cv2.TrackerTLD_create(),
#                 'medianflow': cv2.TrackerMedianFlow_create(),
#                 'goturn': cv2.TrackerGOTURN_create(),
#                 'mosse': cv2.TrackerMOSSE_create(),
#                 'csrt': cv2.TrackerCSRT_create()}
#                 # 're3': re3_tracker.Re3Tracker(),
#                 # 'pysot': base_tracker.BaseTracker()}
        
trackers = {'medianflow': cv2.TrackerMedianFlow_create()}

In [ ]:
# clips = {'Bike02': '/Users/kierandonnelly/thesis/raw_clips/Bike02.mp4',
#         'Bike03': '/Users/kierandonnelly/thesis/raw_clips/Bike03.mp4',
#         'Bike04': '/Users/kierandonnelly/thesis/raw_clips/Bike04.mp4',
#         'Bike05': '/Users/kierandonnelly/thesis/raw_clips/Bike05.mp4',
#         'Bike07': '/Users/kierandonnelly/thesis/raw_clips/Bike07.mp4',
#         'Bike08': '/Users/kierandonnelly/thesis/raw_clips/Bike08.mp4',
#         'Bike09': '/Users/kierandonnelly/thesis/raw_clips/Bike09.mp4',
#         'Ski01': '/Users/kierandonnelly/thesis/raw_clips/Ski01.mp4',
#         'Ski02': '/Users/kierandonnelly/thesis/raw_clips/Ski02.mp4',
#         'Ski03': '/Users/kierandonnelly/thesis/raw_clips/Ski03.mp4',
#         'Ski04': '/Users/kierandonnelly/thesis/raw_clips/Ski04.mp4',
#         'Snowboard01': '/Users/kierandonnelly/thesis/raw_clips/Snowboard01.mp4',
#         'Sup01': '/Users/kierandonnelly/thesis/raw_clips/Sup01.mp4',
#         'Surf01': '/Users/kierandonnelly/thesis/raw_clips/Surf01.mp4',
#         'Wake01': '/Users/kierandonnelly/thesis/raw_clips/Wake01.mp4',
#         'Car01': '/Users/kierandonnelly/thesis/raw_clips/Car01.mp4',
#         'Human01': '/Users/kierandonnelly/thesis/raw_clips/Human01.mp4',
#         'Slalom01': '/Users/kierandonnelly/thesis/raw_clips/Slalom01.mp4'}

clips = {'Sup01': '/Users/kierandonnelly/thesis/raw_clips/Sup01.mp4',
        'Snowboard01': '/Users/kierandonnelly/thesis/raw_clips/Snowboard01.mp4'}

In [ ]:
# resize = (1,2,4)

# resize = [0, 1280, 640]

resize = [0]

### Iterate over videos in directory and run tracker

In [ ]:
metrics_list = []

# iterate over clips
for clip_name, clip_loc in clips.items(): 
    print('[INFO] Clip name: ' + str(clip_name))
    
    # retrieve frame data for clip
    frames = pd.read_csv('./frame_data/' + str(clip_name) + '.csv', sep=';')
    
    # iterate over tracker dictionary
    for tracker_name, tracker_inst in trackers.items():
        
        for i in resize:
            # call tracking function
            metrics = run_track(tracker_name, tracker_inst, clip_name, clip_loc, frames, i)
            
            metrics_list.append(metrics)
    
# convert list of metrics dicts to df
results_df = pd.DataFrame(metrics_list, columns=['Clip', 'Res', 'Tracker', 'SOTP', 'SOTA', 'Recall',\
                                                 'Precision', 'FPF', 'SODP', 'MT', 'PT', 'ML', 'Completeness',\
                                                 'FM', 'FPS', 'fp_count', 'fp_thresh_count', 'tp_count',\
                                                 'fn_count', 'tn_count', 'N_objects', 'N_frames', 'N_match'])

# save to CSV
results_df.to_csv('metrics_' + time.strftime('%l:%M:%S' + '.csv'), index=False)

## Testing